# Calibrating the model

I generated calibration data in R, in code/cleaning/clean_BEA_calibration.R. I compute our exogenous variables as...

$$
\begin{aligned}
\frac{P_j X_{ij}}{P_i Y_i} = \omega_{ij} \tilde{Z}^{\theta- 1} \left(\frac{P_{j}}{Q_i}\right)^{1-\theta} \rightarrow \omega_{ij} = \frac{\bar{X}_{ij}}{\bar{Y}_i}, \text{ expenditure share at base year, normalized to sum to 1} \\
\frac{W_i L_i}{P_i Y_i} = \alpha_i Z^{\epsilon - 1} \left(\frac{W_i}{P_i}\right)^{1-\epsilon} \rightarrow \alpha_i = \frac{\bar{L}_i}{\bar{Y}_i}, \text{ value-added share at base year} \\
\beta_i = \bar{C}_i = \bar{Y}_i - \sum_j \bar{X}_{ji} = \bar{Y}_i - \sum_j \bar{Y}_j (1-\alpha_j) \omega_{ji} = \mathbf{Y}(\mathbf{I} - \text{diag}(1-\alpha)\Omega), \text{ normalized to sum to 1} \\
\bar{L}_i = \alpha_i \bar{Y}_i = \alpha \times (\mathbf{I} - \text{diag}(1-\alpha)\Omega)^{-1} \\
\end{aligned}
$$

By default, I use 2023 as the base year.

In [2]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# I load calibration_data from R
ro.r['load']('../../data/cleaned/structural/calibration_data.RData')
ro.r['load']('../../data/cleaned/code_desc_crosswalk.RData')

# Convert R data frames to pandas DataFrames
with localconverter(ro.default_converter + pandas2ri.converter):
    code_desc_crosswalk = ro.conversion.rpy2py(ro.r['code_desc_crosswalk'])
    Y_list = ro.conversion.rpy2py(ro.r['Y_list'])
    Omega_list = ro.conversion.rpy2py(ro.r['Omega_list'])
    alpha_list = ro.conversion.rpy2py(ro.r['alpha_list'])
    beta_list = ro.conversion.rpy2py(ro.r['beta_list'])
    L_list = ro.conversion.rpy2py(ro.r['L_list'])
    industry_tfp = ro.conversion.rpy2py(ro.r['industry_TFP'])
    elasticity_byyear = ro.conversion.rpy2py(ro.r['elasticity_byyear'])
    elasticity_bycode = ro.conversion.rpy2py(ro.r['elasticity_byCode'])
    cumulative_delta_logOmega = ro.conversion.rpy2py(ro.r['cumulative_delta_logOmega_wide']).to_numpy()
    delta_logOmega_list = ro.conversion.rpy2py(ro.r['delta_logOmega_wide_list'])
    delta_logOmega_predicted10 = ro.conversion.rpy2py(ro.r['delta_logOmega_predicted10']).to_numpy()

# generate the covariance matrix of the industry TFP
industry_tfp_annual = industry_tfp.pivot(index='Code', columns='year', values='delta_tfp_1').to_numpy()
industry_tfp_vcov_annual = np.cov(industry_tfp_annual)
industry_tfp_quadrennial = industry_tfp.pivot(index='Code', columns='year', values='delta_tfp_4').to_numpy()
industry_tfp_vcov_quadrennial = np.cov(industry_tfp_quadrennial)

# set off diagonal elements to zero (assume uncorrelated shocks)
industry_tfp_vcov_annual = np.diag(np.diag(industry_tfp_vcov_annual))
industry_tfp_vcov_quadrennial = np.diag(np.diag(industry_tfp_vcov_quadrennial))

# Solving for equilibrium

I solve for two different types of equilibrium: one where labor is not allowed to reallocate across sectors, thereby exogenously fixing $L_i$, and one where labor is fully allowed to reallocate across sectors, equating sectoral wages $W_i = W_j = W$ for all $i,j$. 

## No reallocation

An equilibrium is defined as wages $\{W_i\}_{i=0}^N$, prices $\{P_i\}_{i=0}^N$ and quantities ${Y_i}$ such that...

1) Wages are equal to the marginal product of labor, i.e. $W_i = P_i \times \partial F_i/\partial L_i$ for all $i$. 
2) Prices are equal to marginal cost, i.e. $P_i = MC_i$ for all $i$.
3) All goods markets clear, i.e. $Y_i = \sum_j X_{ij} + C_i$. 

In practice, wages are perfectly pinned down by $Y,P$ and the exogenous variables. So we are solving for prices $P$ and quantities $Y$ that satisfy (2) and (3). We need to write these constraints in terms of exogenous parameters and givens, which we get via the FOC of the firm and household problems. Note that the price of the consumption bundle $\left(\sum_j \beta_j p_j^{1-\sigma}\right)^{\frac{1}{1-\sigma}}$ is normalized to 1. 

$$
\begin{aligned}
MC_i = A_i^{\epsilon_i - 1} (\alpha_i w_i^{1-\epsilon_i}+ (1-\alpha_i)q_i^{1-\epsilon_i})^{\frac{1}{1-\epsilon_i}} \\
X_{ij} = y_i (1-\alpha_i) \omega_{ij} p_i^{\epsilon_i} p_j^{-\theta_i} q_i^{\theta_i-\epsilon_i}Z_i^{\epsilon_i - 1} \tilde{Z}_i^{\theta_i - 1} \\
C_i = C \beta_i p_i^{-\sigma} = wL \beta_i p_i^{-\sigma} 
\end{aligned}
$$

## Full reallocation

Same definition as above, but now labor is endogenously determined, and wages are equalized across sectors. I normalize the wage $W = 1$ to be the numeraire (whereas before the consumption bundle price was normalized to 1), and I redefine aggregate consumption as $C = \sum_i \beta_i p_i^{1-\sigma}$. Other than that things pass as normal.

Note that prices are written using lower cases, since they are defined in relation to the normalization year. 

## Functions

I define functions...

- ```xij(A, Omega, alpha, epsilon, theta, Q, Y, P)```: computes the matrix of intermediate input demand $\{x_{ij}\}$ given exogenous parameters and guesses for prices and quantities.
- ```multisector_constraints_realloc(X, A, Omega, alpha, epsilon, theta, Q, Y, P, C, sigma)```: returns a vector of $P_i - MC_i$ and $Y_i - \sum_j x_{ij} - c_i$ for all $i$ given guesses of prices and quantities in $X$; these need to go to 0 in eqm.
- ```multisector_constraints_no_realloc(X, A, Omega, alpha, epsilon, theta, Q, Y, P, C, sigma)```: same as above, but with labor fixed.
- ```trivial(X)```: pure place-holder, arbitrary function of guess so I can use scipy.optimize.minimize with constraints.
- ```eqm_solver(A, Omega, alpha, epsilon, theta, Q, Y, P, C, sigma, realloc, return_L)```: solves for equilibrium prices and quantities given exogenous parameters and a boolean for whether labor is allowed to reallocate and whether to return labor.
- ```draw_multivariate_normal(cov, num_samples, seed)```: draws from a multivariate normal with a given covariance matrix and seed.
- ```eqm_simulator(A, Omega, alpha, epsilon, theta, Q, Y, P, C, sigma, realloc, return_L, num_samples, seed)```: solves for equilibria given exogenous parameters and TFP shocks drawn from a multivariate normal. Returns GDP and sectoral output.
- ```CD_simulator(A, Omega, alpha, epsilon, theta, Q, Y, P, C, sigma, realloc, return_L, num_samples, seed)```: solves for equilibria but using Cobb-Douglas production functions.

# Elasticity exercises

Exercises examining how my estimates of heterogenous elasticities, across years and across sectors, change the predicted macro effect of sectoral shocks.

## Time-varying elasticity exercise 

Generate realistic shocks calibrated to TFP data (annual and quadrennial). Compare equilibrium GDP distributions with high/low elasticity estimates (p75, p25), along with Cobb-Douglas benchmark.

In [15]:
# load solver functions
from eqm_solver_functions import *

# set default values (2023)
year = '2023'
Omega = Omega_list[year]
alpha = alpha_list[year].flatten()
beta = beta_list[year].flatten()
L = L_list[year].flatten()

# exogenous elasticities
epsilon = 0.6 # across VA and II; Alireza+ 
sigma = 0.7

# load elasticities
theta_high = elasticity_bycode['p75_theta'].to_numpy()
theta_low = elasticity_bycode['p25_theta'].to_numpy()

# set initial guesses
Y_norm = beta@np.linalg.inv(np.eye(len(alpha)) - np.diag(1 - alpha)@Omega)
P_norm = np.ones(len(alpha))

# get Domar weights; used for Cobb-Douglas comparison
domar_weights = beta@np.linalg.inv(np.eye(len(alpha)) - np.diag(1 - alpha)@Omega)

# draw shocks
annual_shocks = draw_multivariate_normal(industry_tfp_vcov_annual, 1000, seed=1)
quad_shocks = draw_multivariate_normal(industry_tfp_vcov_quadrennial, 1000, seed=1) # magnified shocks = business-cycle shocks

# simulate GDP and sectoral output distributions
gdp_dist_CD_annual = CD_simulator(domar_weights, annual_shocks)
gdp_dist_high_annual, sectoral_output_high_annual = eqm_simulator(beta, Omega, alpha, epsilon, theta_high, sigma, L, P_norm, Y_norm, annual_shocks)
gdp_dist_low_annual, sectoral_output_low_annual = eqm_simulator(beta, Omega, alpha, epsilon, theta_low, sigma, L, P_norm, Y_norm, annual_shocks)
gdp_dist_CD_quad = CD_simulator(domar_weights, quad_shocks)
gdp_dist_high_quad, sectoral_output_high_quad = eqm_simulator(beta, Omega, alpha, epsilon, theta_high, sigma, L, P_norm, Y_norm, quad_shocks)
gdp_dist_low_quad, sectoral_output_low_quad = eqm_simulator(beta, Omega, alpha, epsilon, theta_low, sigma, L, P_norm, Y_norm, quad_shocks)

# save results
gdp_dist = pd.DataFrame({'gdp_dist_CD_annual': gdp_dist_CD_annual, 'gdp_dist_high_annual': gdp_dist_high_annual, 'gdp_dist_low_annual': gdp_dist_low_annual, 'gdp_dist_CD_quad': gdp_dist_CD_quad, 'gdp_dist_high_quad': gdp_dist_high_quad, 'gdp_dist_low_quad': gdp_dist_low_quad})
gdp_dist.to_csv("../../data/cleaned/structural/simulated_gdp.csv", index=False)

Convergence!
Remaining samples:  999
Convergence!
Remaining samples:  998
Convergence!
Remaining samples:  997
Convergence!
Remaining samples:  996
Convergence!
Remaining samples:  995
Convergence!
Remaining samples:  994
Convergence!
Remaining samples:  993
Convergence!
Remaining samples:  992
Convergence!
Remaining samples:  991
Convergence!
Remaining samples:  990
Convergence!
Remaining samples:  989
Convergence!
Remaining samples:  988
Convergence!
Remaining samples:  987
Convergence!
Remaining samples:  986
Convergence!
Remaining samples:  985
Convergence!
Remaining samples:  984
Convergence!
Remaining samples:  983
Convergence!
Remaining samples:  982
Convergence!
Remaining samples:  981
Convergence!
Remaining samples:  980
Convergence!
Remaining samples:  979
Convergence!
Remaining samples:  978
Convergence!
Remaining samples:  977
Convergence!
Remaining samples:  976
Convergence!
Remaining samples:  975
Convergence!
Remaining samples:  974
Convergence!
Remaining samples:  973
C

In [3]:
from scipy.stats import skew
import pandas as pd
import numpy as np

# import gdp_dist
gdp_dist = pd.read_csv("../../data/cleaned/structural/simulated_gdp.csv")
gdp_dist = gdp_dist.dropna() # remove rows with nans
gdp_dist_CD_annual = gdp_dist['gdp_dist_CD_annual']
gdp_dist_high_annual = gdp_dist['gdp_dist_high_annual']
gdp_dist_low_annual = gdp_dist['gdp_dist_low_annual']
gdp_dist_CD_quad = gdp_dist['gdp_dist_CD_quad']
gdp_dist_high_quad = gdp_dist['gdp_dist_high_quad']
gdp_dist_low_quad = gdp_dist['gdp_dist_low_quad']

# Calculate statistics for annual shocks
annual_stats = {
    'mean': [np.mean(np.log(gdp_dist_CD_annual)), np.mean(np.log(gdp_dist_high_annual)), np.mean(np.log(gdp_dist_low_annual))],
    'sd': [np.std(np.log(gdp_dist_CD_annual)), np.std(np.log(gdp_dist_high_annual)), np.std(np.log(gdp_dist_low_annual))],
    'skew': [skew(np.log(gdp_dist_CD_annual)), skew(np.log(gdp_dist_high_annual)), skew(np.log(gdp_dist_low_annual))]
}

# Calculate statistics for quad shocks
quad_stats = {
    'mean': [np.mean(np.log(gdp_dist_CD_quad)), np.mean(np.log(gdp_dist_high_quad)), np.mean(np.log(gdp_dist_low_quad))],
    'sd': [np.std(np.log(gdp_dist_CD_quad)), np.std(np.log(gdp_dist_high_quad)), np.std(np.log(gdp_dist_low_quad))],
    'skew': [skew(np.log(gdp_dist_CD_quad)), skew(np.log(gdp_dist_high_quad)), skew(np.log(gdp_dist_low_quad))]
}

# Create a DataFrame for the results

results_baseline = pd.DataFrame({
    'Cobb-Douglas': [annual_stats['mean'][0], annual_stats['sd'][0], annual_stats['skew'][0]],
    'High elasticity': [annual_stats['mean'][1], annual_stats['sd'][1], annual_stats['skew'][1]],
    'Low elasticity': [annual_stats['mean'][2], annual_stats['sd'][2], annual_stats['skew'][2]]
}, index=['Mean', 'SD', 'Skewness'])

results_quad = pd.DataFrame({
    'Cobb-Douglas': [quad_stats['mean'][0], quad_stats['sd'][0], quad_stats['skew'][0]],
    'High elasticity': [quad_stats['mean'][1], quad_stats['sd'][1], quad_stats['skew'][1]],
    'Low elasticity': [quad_stats['mean'][2], quad_stats['sd'][2], quad_stats['skew'][2]],
}, index=['Mean', 'SD', 'Skewness'])

# save table to latex 
results_baseline = results_baseline.transpose()
results_quad = results_quad.transpose()
results = pd.concat([results_baseline, results_quad], axis=1)
results_tab = results.to_latex(float_format = '%.3f')
custom_header = r"""
\begin{tabular}{lcccccc}
\toprule
& \multicolumn{3}{c}{Annual shocks} & \multicolumn{3}{c}{Quadrennial shocks} \\
"""
results_tab = results_tab.replace(r"\begin{tabular}{lrrrrrr}", custom_header)
with open("../../tables/shock_simulation_results_combined.tex", "w") as f:
    f.write(results_tab)

## Sector-varying elasticity exercises

Generate severe shocks to individual sectors that cut their output by 75%. Compare resulting equilibrium GDP with sector-varying elasticities versus the corresponding mean imposed uniformly.

In [3]:
# load solver functions
from eqm_solver_functions import *

# set default values (2023)
year = '2023'
Omega = Omega_list[year]
alpha = alpha_list[year].flatten()
beta = beta_list[year].flatten()
L = L_list[year].flatten()

# exogenous elasticities
epsilon = 0.6 # across VA and II; Alireza+ 
sigma = 0.7

# load elasticities
theta_het = elasticity_bycode['mean_theta'].to_numpy()
theta_mean = elasticity_bycode['mean_theta'].mean() * np.ones(len(alpha))

Y_norm = beta@np.linalg.inv(np.eye(len(alpha)) - np.diag(1 - alpha)@Omega)
P_norm = np.ones(len(alpha))

# take SD across all observations in industry_tfp_annual
industry_tfp_sd = np.std(industry_tfp_annual)
outlier = -np.log(2)
extreme_shocks = []
for i in range(len(alpha)): 
    shock = np.zeros(len(alpha))
    shock[i] = outlier
    extreme_shocks.append(shock)

gdp_dist_het_extreme, sectoral_output_het_extreme = eqm_simulator(beta, Omega, alpha, epsilon, theta_het, sigma, L, P_norm, Y_norm, extreme_shocks)
gdp_dist_mean_extreme, sectoral_output_mean_extreme = eqm_simulator(beta, Omega, alpha, epsilon, theta_mean, sigma, L, P_norm, Y_norm, extreme_shocks)
gdp_diff = np.log(gdp_dist_het_extreme) - np.log(gdp_dist_mean_extreme) # log deviation from non-stochastic mean

# dataframe with industry labels
gdp_diff = pd.DataFrame({'Het.': np.log(gdp_dist_het_extreme), 'Uniform': np.log(gdp_dist_mean_extreme), 'Difference': gdp_diff, 'Code': elasticity_bycode['Code']})
gdp_diff = gdp_diff.merge(code_desc_crosswalk, on='Code')
# make table of industry description and smallest differences
diff_tab = gdp_diff.sort_values(by='Het.') 
diff_tab = diff_tab[['Industry Description', 'Het.', 'Uniform', 'Difference']]
diff_tab = diff_tab.head(3)
# rename columns
diff_tab.columns = ['Industry Description', '$\\Delta$ GDP (Het.)', '$\\Delta$ GDP (Unif.)', 'Difference']
diff_tab.to_latex("../../tables/industry_diff_extreme.tex", index=False, float_format = '%.3f', column_format='lccc')

/Users/jacobgosselin/Documents(local)/GitHub/HeterogeousSectoralProduction/code/quantitative_exc/eqm_solver_functions.py:110: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', True, aligned=True), Array(float64, 1, 'C', False, aligned=True))
  'fun': jit(lambda X: multisector_constraints_norealloc(X, A, beta, Omega, alpha, epsilon, theta, sigma, L))


Convergence!
Remaining samples:  65
Convergence!
Remaining samples:  64
Convergence!
Remaining samples:  63
Convergence!
Remaining samples:  62
Convergence!
Remaining samples:  61
Convergence!
Remaining samples:  60
Convergence!
Remaining samples:  59
Convergence!
Remaining samples:  58
Convergence!
Remaining samples:  57
Convergence!
Remaining samples:  56
Convergence!
Remaining samples:  55
Convergence!
Remaining samples:  54
Convergence!
Remaining samples:  53
Convergence!
Remaining samples:  52
Convergence!
Remaining samples:  51
Convergence!
Remaining samples:  50
Convergence!
Remaining samples:  49
Convergence!
Remaining samples:  48
Convergence!
Remaining samples:  47
Convergence!
Remaining samples:  46
Convergence!
Remaining samples:  45
Convergence!
Remaining samples:  44
Convergence!
Remaining samples:  43
Convergence!
Remaining samples:  42
Convergence!
Remaining samples:  41
Convergence!
Remaining samples:  40
Convergence!
Remaining samples:  39
Convergence!
Remaining sampl

# Share parameter exercises

## Share parameters and changes in sectoral output

Using the cumulative sum of share parameter shifts estimated empirically, simulate counterfactual Omega corresponding to 2023 (assuming no other exogenous parameters change). Estimate response of sectoral output, with and without labor reallocation.

In [12]:
# set initial values for 1997
year = '1997'
Omega = Omega_list[year]
alpha = alpha_list[year].flatten()
beta = beta_list[year].flatten()
L = L_list[year].flatten()

# exogenous elasticities
epsilon = 0.6 # across VA and II; Alireza+ 
sigma = .7

# load elasticities
theta = elasticity_bycode['theta'].to_numpy()
theta_mean = elasticity_byyear['theta'].mean() * np.ones(len(alpha))

# get counterfactual Omega
Omega_counterfactual = Omega.copy()
delta_Omega = np.exp(cumulative_delta_logOmega)
Omega_counterfactual = Omega_counterfactual * delta_Omega

Omega_counterfactual = Omega_counterfactual / Omega_counterfactual.sum(axis=1)[:, np.newaxis] # normalize rows to sum to 1

Y_norm = beta@np.linalg.inv(np.eye(len(alpha)) - np.diag(1 - alpha)@Omega)
P_norm = np.ones(len(alpha))

# solve for counterfactual output without reallocation
C_0, Y_0 = eqm_solver(np.ones(len(alpha)), beta, Omega, alpha, epsilon, theta, sigma, L, P_norm, Y_norm)
C_1, Y_1 = eqm_solver(np.ones(len(alpha)), beta, Omega_counterfactual, alpha, epsilon, theta, sigma, L, P_norm, Y_norm)
counterfactual_output = pd.DataFrame({'Code': elasticity_bycode['Code'], 'Y_1': Y_1, 'Y_0': Y_0, 'log_change': np.log(Y_1) - np.log(Y_0)})
counterfactual_output = counterfactual_output.sort_values(by='log_change', ascending=False)

# solve for counterfactual output with reallocation
C_0, Y_0 = eqm_solver(np.ones(len(alpha)), beta, Omega, alpha, epsilon, theta, sigma, L, P_norm, Y_norm, reallocate=True)
C_1, Y_1 = eqm_solver(np.ones(len(alpha)), beta, Omega_counterfactual, alpha, epsilon, theta, sigma, L, P_norm, Y_norm, reallocate=True)
counterfactual_output_realloc = pd.DataFrame({'Code': elasticity_bycode['Code'], 'Y_1': Y_1, 'Y_0': Y_0, 'log_change': np.log(Y_1) - np.log(Y_0)})
counterfactual_output_realloc = counterfactual_output_realloc.sort_values(by='log_change', ascending=False)

# select 3 largest increases in output, without and with reallocation
counterfactual_output_combined_table = pd.concat([counterfactual_output.head(3), counterfactual_output_realloc.head(3)])
counterfactual_output_combined_table = counterfactual_output_combined_table.merge(code_desc_crosswalk, on='Code')
counterfactual_output_combined_table = counterfactual_output_combined_table[['Industry Description', 'log_change']]
# abridge industry descriptions at 50 characters
# add ellipsis if longer
counterfactual_output_combined_table['Industry Description'] = counterfactual_output_combined_table['Industry Description'].apply(lambda x: x[:50] + '...' if len(x) > 50 else x)
counterfactual_output_combined_table = counterfactual_output_combined_table.rename(columns={'log_change': 'Log Change in Output'})

latex_table = counterfactual_output_combined_table.to_latex(index=False, float_format="%.3f")
lines = latex_table.splitlines()
lines.insert(4, r'\midrule')
lines.insert(5, r'\multicolumn{2}{l}{\textbf{No labor reallocation}} \\')
lines.insert(6, r'\midrule')
# add a blank line
lines.insert(10, r'\midrule')
lines.insert(11, r'\multicolumn{2}{l}{\textbf{Full labor reallocation}} \\')
lines.insert(12, r'\midrule')
# lines.insert(3, r'\midrule')
# lines.insert(4, r'\multicolumn{2}{c}{\textbf{No labor reallocation}} \\')
# lines.insert(5, r'\midrule')
latex_table = '\n'.join(lines)

with open("../../tables/cumulative_counterfactual_output_change.tex", "w") as f:
    f.write(latex_table)

Convergence!
Convergence!
Convergence!
Convergence!


Use year-to-year changes in share parameters, predict the effect on sectoral output. Save results to compare to actual data.

In [ ]:
# exogenous elasticities
epsilon = 0.6 # across VA and II; Alireza+ 
sigma = .7

# load elasticities
theta = elasticity_bycode['mean_theta'].to_numpy()

# initial guesses
Y_norm = beta@np.linalg.inv(np.eye(len(alpha)) - np.diag(1 - alpha)@Omega)
P_norm = np.ones(len(alpha))

# set initial Omega
temp_Omega = Omega.copy()

# initialize counterfactual sectoral output
counterfactual_output_change = pd.DataFrame({'Code': elasticity_bycode['Code']})

# set starting point 
C_previous, Y_previous = eqm_solver(np.ones(len(alpha)), beta, Omega, alpha, epsilon, theta, sigma, L, P_norm, Y_norm) 

for i in range(1997, 2023):
    # t = 0 calibration
    temp_Omega = Omega_list[str(i)]
    temp_alpha = alpha_list[str(i)].flatten()
    temp_beta = beta_list[str(i)].flatten()
    temp_L = L_list[str(i)].flatten()

    # t = 1 change
    Omega_counterfactual = temp_Omega.copy()
    delta_Omega = np.exp(delta_logOmega_list[str(i+1)]).to_numpy()
    Omega_counterfactual = Omega_counterfactual * delta_Omega
    Omega_counterfactual = Omega_counterfactual / Omega_counterfactual.sum(axis=1)[:, np.newaxis] # normalize to 1

    # corresponding beta_counterfactual, L_counterfactual
    Y = Y_list[str(i)]
    beta_counterfactual = Y@(np.eye(len(alpha)) - np.diag(alpha))@Omega_counterfactual
    beta_counterfactual = beta_counterfactual / beta_counterfactual.sum()
    L_counterfactual = alpha*(beta_counterfactual.T@np.linalg.inv(np.eye(len(alpha)) - (np.diag(1 - alpha)@Omega_counterfactual)))


    # solve for C_0, Y_0, C_1, Y_1
    C_0, Y_0 = eqm_solver(np.ones(len(alpha)), temp_beta, temp_Omega, temp_alpha, epsilon, theta, sigma, temp_L, P_norm, Y_norm)
    C_1, Y_1 = eqm_solver(np.ones(len(alpha)), temp_beta, Omega_counterfactual, temp_alpha, epsilon, theta, sigma, temp_L, P_norm, Y_norm)
    counterfactual_output_change[str(i+1)] = np.log(Y_1) - np.log(Y_0)

counterfactual_output_change.to_csv("../../data/cleaned/structural/counterfactual_output_change_byyear.csv", index=False)


## Effect on sectoral shocks 

Using the counterfactual $\tilde{\Omega}_{2023}$ corresponding to share parameter shifts between 1997-2023, estimate the effect of severe shocks to individual sectors on GDP. Note that I list log-deviations from non-shocked GDP; since log output at the original point is 0, this doesn't matter, but it does for the counterfactual Omega, since that implies a different non-shocked GDP.

In [4]:
from eqm_solver_functions import *

# set initial values for 1997
year = '1997'
Omega = Omega_list[year]
alpha = alpha_list[year].flatten()
beta = beta_list[year].flatten()
L = L_list[year].flatten()

# exogenous elasticities
epsilon = 0.6 # across VA and II; Alireza+ 
sigma = .7

# load elasticities
theta = elasticity_bycode['mean_theta'].to_numpy()

# get counterfactual Omega
Omega_counterfactual = Omega.copy()
Omega_counterfactual = Omega_counterfactual * np.exp(cumulative_delta_logOmega)
Omega_counterfactual = Omega_counterfactual / Omega_counterfactual.sum(axis=1)[:, np.newaxis] # normalize rows to sum to 1

# set initial guesses
Y_norm = beta@np.linalg.inv(np.eye(len(alpha)) - np.diag(1 - alpha)@Omega)
P_norm = np.ones(len(alpha))

# recover reallocated L
L_counterfactual = eqm_solver(np.ones(len(alpha)), beta, Omega_counterfactual, alpha, epsilon, theta, sigma, L, P_norm, Y_norm, reallocate=True, return_L=True)

# extreme shocks
outlier = -np.log(2) # cut output by 75%
extreme_shocks = []
for i in range(len(alpha)): 
    shock = np.zeros(len(alpha))
    shock[i] = outlier
    extreme_shocks.append(shock)

gdp_noshock_0, sectoral_output_noshock_0 = eqm_solver(np.ones(len(alpha)), beta, Omega, alpha, epsilon, theta, sigma, L, P_norm, Y_norm)
gdp_dist_0, sectoral_output_0 = eqm_simulator(beta, Omega, alpha, epsilon, theta, sigma, L, P_norm, Y_norm, extreme_shocks)
gdp_dist_0 = np.log(gdp_dist_0) - np.log(gdp_noshock_0) # dataframe difference in gdp, shock - no shock
gdp_noshock_1, sectoral_output_noshock_1 = eqm_solver(np.ones(len(alpha)), beta, Omega_counterfactual, alpha, epsilon, theta, sigma, L_counterfactual, P_norm, Y_norm)
gdp_dist_1, sectoral_output_1= eqm_simulator(beta, Omega_counterfactual, alpha, epsilon, theta, sigma, L_counterfactual, P_norm, Y_norm, extreme_shocks)
gdp_dist_1 = np.log(gdp_dist_1) - np.log(gdp_noshock_1) # dataframe difference in gdp, shock - no shock
gdp_diff = np.abs(gdp_dist_1) - np.abs(gdp_dist_0) # magnitude difference 

gdp_diff = pd.DataFrame({'2023':gdp_dist_1, '1997':gdp_dist_0, 'Difference': gdp_diff, 'Code': elasticity_bycode['Code']}) # dataframe with industry labels
gdp_diff = gdp_diff.merge(code_desc_crosswalk, on='Code')
gdp_diff = gdp_diff[['Industry Description', '2023', '1997', 'Difference']] 
gdp_diff['Industry Description'] = gdp_diff['Industry Description'].apply(lambda x: x[:50] + '...' if len(x) > 50 else x) # abridge industry descriptions at 50 characters

diff_tab_decrease = gdp_diff.sort_values(by='Difference', ascending=True).head(3)
diff_tab_increase = gdp_diff.sort_values(by='Difference', ascending=False).head(3)
diff_tab = pd.concat([diff_tab_increase, diff_tab_decrease]) # stack dataframes
diff_tab = diff_tab[['Industry Description', 'Difference']] # select columns
diff_tab = diff_tab.rename(columns={'Difference': '$\\lvert \\Delta \\text{GDP}(\\tilde{\\Omega}_{2023}) \\rvert - \\lvert \\Delta \\text{GDP}(\\Omega_{1997}) \\rvert$'}) # rename columns

# write table
latex_table = diff_tab.to_latex(index=False, float_format="%.3f", column_format='lccc')
lines = latex_table.splitlines()
lines.insert(4, r'\midrule')
lines.insert(5, r'\multicolumn{2}{l}{\textbf{Largest increases in GDP effect}} \\')
lines.insert(6, r'\midrule')
# add a blank line
lines.insert(10, r'\midrule')
lines.insert(11, r'\multicolumn{2}{l}{\textbf{Largest decreases in GDP effect}} \\')
lines.insert(12, r'\midrule')
latex_table = '\n'.join(lines)

with open("../../tables/ind_sector_shock_1997_2023.tex", "w") as f:
    f.write(latex_table)

Convergence!
Convergence!
Convergence!
Remaining samples:  65
Convergence!
Remaining samples:  64
Convergence!
Remaining samples:  63
Convergence!
Remaining samples:  62
Convergence!
Remaining samples:  61
Convergence!
Remaining samples:  60
Convergence!
Remaining samples:  59
Convergence!
Remaining samples:  58
Convergence!
Remaining samples:  57
Convergence!
Remaining samples:  56
Convergence!
Remaining samples:  55
Convergence!
Remaining samples:  54
Convergence!
Remaining samples:  53
Convergence!
Remaining samples:  52
Convergence!
Remaining samples:  51
Convergence!
Remaining samples:  50
Convergence!
Remaining samples:  49
Convergence!
Remaining samples:  48
Convergence!
Remaining samples:  47
Convergence!
Remaining samples:  46
Convergence!
Remaining samples:  45
Convergence!
Remaining samples:  44
Convergence!
Remaining samples:  43
Convergence!
Remaining samples:  42
Convergence!
Remaining samples:  41
Convergence!
Remaining samples:  40
Convergence!
Remaining samples:  39
Co

Now I do the same exercise, but the baseline year is 2023, and I generate $\tilde{\Omega}_{2033}$ to predict future share parameters based on linear trends (i.e. add the share parameter shifts from 2013-2023).

In [6]:
# set initial values for 2023
year = '2023'
Omega = Omega_list[year]
alpha = alpha_list[year].flatten()
beta = beta_list[year].flatten()
L = L_list[year].flatten()

# exogenous elasticities
epsilon = 0.6 # across VA and II; Alireza+ 
sigma = .7

# load elasticities
theta = elasticity_bycode['mean_theta'].to_numpy()

# get predicted Omega
Omega_predicted = Omega.copy()
Omega_predicted = Omega_predicted * np.exp(delta_logOmega_predicted10)
Omega_predicted = Omega_predicted / Omega_predicted.sum(axis=1)[:, np.newaxis] # normalize rows to sum to 1

# set initial guesses
Y_norm = beta@np.linalg.inv(np.eye(len(alpha)) - np.diag(1 - alpha)@Omega)
P_norm = np.ones(len(alpha))

# recover reallocated L
L_counterfactual = eqm_solver(np.ones(len(alpha)), beta, Omega_predicted, alpha, epsilon, theta, sigma, L, P_norm, Y_norm, reallocate=True, return_L=True)

# extreme shocks
outlier = -np.log(2) # cut output by 75%
extreme_shocks = []
for i in range(len(alpha)): 
    shock = np.zeros(len(alpha))
    shock[i] = outlier
    extreme_shocks.append(shock)

gdp_noshock_0, sectoral_output_noshock_0 = eqm_solver(np.ones(len(alpha)), beta, Omega, alpha, epsilon, theta, sigma, L, P_norm, Y_norm)
gdp_dist_0, sectoral_output_0 = eqm_simulator(beta, Omega, alpha, epsilon, theta, sigma, L, P_norm, Y_norm, extreme_shocks)
gdp_dist_0 = np.log(gdp_dist_0) - np.log(gdp_noshock_0) # dataframe difference in gdp, shock - no shock
gdp_noshock_1, sectoral_output_noshock_1 = eqm_solver(np.ones(len(alpha)), beta, Omega_predicted, alpha, epsilon, theta, sigma, L_counterfactual, P_norm, Y_norm)
gdp_dist_1, sectoral_output_1= eqm_simulator(beta, Omega_predicted, alpha, epsilon, theta, sigma, L_counterfactual, P_norm, Y_norm, extreme_shocks)
gdp_dist_1 = np.log(gdp_dist_1) - np.log(gdp_noshock_1) # dataframe difference in gdp, shock - no shock
gdp_diff = np.abs(gdp_dist_1) - np.abs(gdp_dist_0) # magnitude difference 

gdp_diff = pd.DataFrame({'2033':gdp_dist_1, '2023':gdp_dist_0, 'Difference': gdp_diff, 'Code': elasticity_bycode['Code']}) # dataframe with industry labels
gdp_diff = gdp_diff.merge(code_desc_crosswalk, on='Code')
gdp_diff = gdp_diff[['Industry Description', '2033', '2023', 'Difference']] 
gdp_diff['Industry Description'] = gdp_diff['Industry Description'].apply(lambda x: x[:50] + '...' if len(x) > 50 else x) # abridge industry descriptions at 50 characters

diff_tab_decrease = gdp_diff.sort_values(by='Difference', ascending=True).head(3)
diff_tab_increase = gdp_diff.sort_values(by='Difference', ascending=False).head(3)
diff_tab = pd.concat([diff_tab_increase, diff_tab_decrease]) # stack dataframes
diff_tab = diff_tab[['Industry Description', 'Difference']] # select columns
diff_tab = diff_tab.rename(columns={'Difference': '$\\lvert \\Delta \\text{GDP}(\\tilde{\\Omega}_{2033}) \\rvert - \\lvert \\Delta \\text{GDP}(\\Omega_{2023}) \\rvert$'}) # rename columns

# write table
latex_table = diff_tab.to_latex(index=False, float_format="%.3f", column_format='lccc')
lines = latex_table.splitlines()
lines.insert(4, r'\midrule')
lines.insert(5, r'\multicolumn{2}{l}{\textbf{Largest increases in GDP effect}} \\')
lines.insert(6, r'\midrule')
# add a blank line
lines.insert(10, r'\midrule')
lines.insert(11, r'\multicolumn{2}{l}{\textbf{Largest decreases in GDP effect}} \\')
lines.insert(12, r'\midrule')
latex_table = '\n'.join(lines)

with open("../../tables/ind_sector_shock_2023_2033.tex", "w") as f:
    f.write(latex_table)

Convergence!
Convergence!
Convergence!
Remaining samples:  65
Convergence!
Remaining samples:  64
Convergence!
Remaining samples:  63
Convergence!
Remaining samples:  62
Convergence!
Remaining samples:  61
Convergence!
Remaining samples:  60
Convergence!
Remaining samples:  59
Convergence!
Remaining samples:  58
Convergence!
Remaining samples:  57
Convergence!
Remaining samples:  56
Convergence!
Remaining samples:  55
Convergence!
Remaining samples:  54
Convergence!
Remaining samples:  53
Convergence!
Remaining samples:  52
Convergence!
Remaining samples:  51
Convergence!
Remaining samples:  50
Convergence!
Remaining samples:  49
Convergence!
Remaining samples:  48
Convergence!
Remaining samples:  47
Convergence!
Remaining samples:  46
Convergence!
Remaining samples:  45
Convergence!
Remaining samples:  44
Convergence!
Remaining samples:  43
Convergence!
Remaining samples:  42
Convergence!
Remaining samples:  41
Convergence!
Remaining samples:  40
Convergence!
Remaining samples:  39
Co